In [ ]:
!pip install elephas -q

In [1]:
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.functions import window, avg, count
from pyspark.sql import types as T
import pyspark.sql.functions as F
from pyspark.sql.window import Window

# from pyspark.mllib.linalg import Matrix, Matrices, Vectors, SparseVector, DenseVector, VectorUDT
from pyspark.ml.linalg import Matrix, Matrices, Vectors, SparseVector, DenseVector, VectorUDT
from pyspark.mllib.evaluation import RegressionMetrics
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline, Transformer
from pyspark.ml.regression import LinearRegression

from elephas.ml_model import ElephasEstimator, ElephasTransformer
from elephas.spark_model import SparkMLlibModel

import numpy as np

from keras.models import Sequential
from keras.layers import Dense
from keras import backend as K
from keras import optimizers

Using TensorFlow backend.


WARNING


#### Create the spark session

In [2]:
spark = (SparkSession
         .builder
         .appName("Streaming")
         .config('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector_2.11:2.4.1,org.apache.spark:spark-sql-kafka-0-10_2.11:2.4.4')
         .getOrCreate())

#### Read our processed time windows from mongo

In [3]:
df = (spark
         .read
         .format("mongo")
         .option("spark.mongodb.input.uri", "mongodb://165.22.199.122/processed.internal")
         .load()
         .drop('_id')
         .orderBy('window.end'))

df.show()

+--------+-----------------+-------------------+-------------------+--------------------+
|n_tweets|            price|          sentiment|          timestamp|              window|
+--------+-----------------+-------------------+-------------------+--------------------+
|      24|9186.630000000001|0.17804166666666665|2019-11-03 14:44:00|[2019-11-03 14:34...|
|      73|9183.607500000002|0.24112465753424658|2019-11-03 14:46:00|[2019-11-03 14:36...|
|     126|9182.878333333334|0.25534523809523824|2019-11-03 14:48:00|[2019-11-03 14:38...|
|     184|         9181.015| 0.2538385869565217|2019-11-03 14:50:00|[2019-11-03 14:40...|
|     257|         9184.389|0.21677548638132285|2019-11-03 14:52:00|[2019-11-03 14:42...|
|     311|         9186.677|0.21157234726688087|2019-11-03 14:54:00|[2019-11-03 14:44...|
|     312|         9189.179|0.19037852564102548|2019-11-03 14:56:00|[2019-11-03 14:46...|
|     316|           9192.1|0.17361044303797465|2019-11-03 14:58:00|[2019-11-03 14:48...|
|     313|

#### Create a transformer to calculate the price difference and generate the y labels

In [4]:
class PriceDiffTransformer(Transformer):
    """
    Custorm tranformer that calculates the price difference since the last time period
    """
    
    def __init__(self):
        super(PriceDiffTransformer, self).__init__()
        
    def _transform(self, df: DataFrame) -> DataFrame:
        # Define the window function
        window = Window.partitionBy().orderBy('timestamp')

        # Create a price lag of 1 window
        df = df.withColumn('prev_price', F.lag(df.price).over(window))

        # Calculate the price difference
        df = df.withColumn('price_diff', df.price - df.prev_price)
        
        # Y label
        df = df.withColumn('label', F.lag(df.price_diff, -1).over(window))

        # Drop the previous price column
        df = df.drop('prev_price', 'window')
        
        # Drop all nan values (first price)
        df = df.na.drop()

        return df

In [ ]:
price_diff_transformer = PriceDiffTransformer()
df_price_diff = price_diff_transformer.transform(df)
df_price_diff.show(5)

#### Create a transformer to bring all the features to one array

In [5]:
class TimeTransformer(Transformer):
    """
    A custom Transformer which transforms all values to timeseries. This is needed to input it into
    the neural network
    """

    def __init__(self):
        super(TimeTransformer, self).__init__()

    def _transform(self, df: DataFrame) -> DataFrame:
        
        # Create the timeseries. Window 24 minutes and collect the list of variables needed
        df_window = (df
             .groupBy(F.window(df.timestamp, '24 minutes', '2 minutes'))
             .agg(
                 F.collect_list('price_diff'), 
                 F.collect_list('sentiment'), 
                 F.collect_list('n_tweets'),
                 F.max('timestamp').alias('timestamp'),
                 F.last('label').alias('label')))

        # Concatenate all array columns
        df_features = df_window.withColumn('features', 
                    F.concat(
                        F.col('collect_list(price_diff)'), 
                        F.col('collect_list(sentiment)'),
                        F.col('collect_list(n_tweets)')))

        # Make sure all the values are there
        df_features = df_features.where(F.size(F.col('features')) == 36)
        
        # Dropped the left over array columns
        df_features = df_features.drop(
            'window', 
            'collect_list(price_diff)', 
            'collect_list(sentiment)', 
            'collect_list(n_tweets)')

        # Parse the features as vector instead of array (length need to be consistent)
        list_to_vector_udf = F.udf(lambda l: Vectors.dense(l), VectorUDT())

        df_features = df_features.select(
            df_features["label"], 
            df_features["timestamp"], 
            list_to_vector_udf(df_features["features"]).alias("features"))

        return df_features.orderBy('timestamp').drop('timestamp')

In [ ]:
time_transformer = TimeTransformer()
df_time = time_transformer.transform(df_price_diff)
df_time.show()

#### Example of the pipeline without the estimator in the end

In [ ]:
price_diff_transformer = PriceDiffTransformer()
time_transformer = TimeTransformer()

transform_pipeline = Pipeline(stages=[price_diff_transformer, time_transformer]).fit(df)
#df_transform = transform_pipeline.transform(df)
#df_transform.show()

### Linear regression test

In [ ]:
price_diff_transformer = PriceDiffTransformer()
time_transformer = TimeTransformer()
lr_estimator = LinearRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)

lr_pipeline = Pipeline(stages=[price_diff_transformer, time_transformer, lr_estimator]).fit(df)
df_lr = lr_pipeline.transform(df)
df_lr.show()

In [ ]:
pred_rdd = df_lr.rdd.map(lambda p: (p.prediction, p.label)).cache()

In [ ]:
metrics = RegressionMetrics(pred_rdd)
metrics.rootMeanSquaredError

#### Elephas prediction

In [54]:
def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true))) 

def build_model():
    model = Sequential()
    model.add(Dense(64, activation='relu', input_shape=(36,)))
    model.add(Dense(32, activation='relu'))
        
    model.add(Dense(1))
    model.compile(optimizer='adam', loss=root_mean_squared_error)
    
    return model

In [88]:
class ElephantEstimator(ElephasEstimator):
    def _transform(self, df):
        
        features_col=self.getFeaturesCol()
    
        output_col = self.getOutputCol()

        label_col = self.getLabelCol()

        #output_col = "prediction"

        #label_col = "label"

        new_schema = copy.deepcopy(df.schema)

        new_schema.add(StructField(output_col, DoubleType(), True))

        rdd = df.rdd.coalesce(1)

        features = np.asarray(

            #rdd.map(lambda x: from_vector(x.features)).collect())
            rdd.map(lambda x: from_vector(x[features_col])).collect())

            # Note that we collect, since executing this on the rdd would require model serialization once again

        #display(len(features[0]))

        model = model_from_yaml(self.get_keras_model_config())

        model.set_weights(self.weights.value)

        #prediction=model.predict(features)
        #display(prediction)

        predictions = rdd.ctx.parallelize(model.predict(features)).coalesce(1)

        #display(predictions.take(2))

        predictions = predictions.map(lambda x: [float(x)])

        #display(predictions.take(2))

        predictions = predictions.map(lambda x: tuple(x))

        display(rdd.zip(predictions).take(2))

        results_rdd = rdd.zip(predictions).map(lambda x: x[0] + x[1])

        # TODO: Zipping like this is very likely wrong

        # results_rdd = rdd.zip(predictions).map(lambda pair: Row(features=to_vector(pair[0].features),

        #                                        label=pair[0].label, prediction=float(pair[1])))


        #display(results_rdd.take(2))


        results_df = df.sql_ctx.createDataFrame(results_rdd, new_schema)

        results_df = results_df.withColumn(

            output_col, results_df[output_col].cast(DoubleType()))

        results_df = results_df.withColumn(

            label_col, results_df[label_col].cast(DoubleType()))

        return results_df

In [71]:
keras_model = build_model()
keras_model.load_weights('models/keras_weights.hdf5')
keras_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 64)                2368      
_________________________________________________________________
dense_5 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 33        
Total params: 4,481
Trainable params: 4,481
Non-trainable params: 0
_________________________________________________________________


In [89]:
adam = optimizers.Adam(lr=0.01)
opt_conf = optimizers.serialize(adam)

# Initialize SparkML Estimator and set all relevant properties
estimator = ElephantEstimator()
estimator.setFeaturesCol('features')
estimator.setLabelCol('label')
estimator.set_keras_model_config(keras_model.to_yaml())
estimator.set_categorical_labels(False)
# estimator.set_nb_classes(nb_classes)
estimator.set_num_workers(1)
estimator.set_epochs(5) 
estimator.set_batch_size(128)
estimator.set_verbosity(1)
estimator.set_validation_split(0.15)
estimator.set_optimizer_config(opt_conf)
estimator.set_mode('synchronous')
estimator.set_loss('mean_squared_error')
estimator.set_metrics(['mean_squared_error'])

ElephantEstimator_893e51a7f654

In [91]:
model = Pipeline(stages=[price_diff_transformer, time_transformer, estimator]).fit(df)
# df_pred = model.transform(df)
# df_pred.show()

### Streaming test

#### Twitter stream

In [6]:
# Define the timestamp format
timestampFormat = "dd-MM-yyyy HH:mm:ss"

# Create the schema of incoming data
twitter_schema = T.StructType([
    T.StructField('timestamp', T.TimestampType(), False),
    T.StructField('text', T.StringType(), False),
    T.StructField('sentiment', T.DoubleType(), False)
])

In [7]:
# Read kafka stream and subscribe to twitter topic
twitter_stream = (spark.readStream
          .format('kafka')
          .option('kafka.bootstrap.servers', 'kafka:9092')
          .option('startingOffsets', 'latest')
          .option('subscribe', 'twitter')
          .load()
          .select(F.col("key").cast("string"), \
                  F.from_json(F.col("value").cast("string"), twitter_schema, \
                  { "timestampFormat": timestampFormat }).alias("value")))

In [8]:
# Create streaming moving windows
twitter_aggregation = (twitter_stream
                     .select('value.*')
                     .withWatermark('timestamp', '5 seconds')
                     .groupBy(window('timestamp', '10 minutes', '2 minutes'))
                     .agg(avg('sentiment').alias('sentiment'), count('timestamp').alias('n_tweets'))).select(F.col('window.end').alias('timestamp'), F.col('sentiment'), F.col('n_tweets'))

In [9]:
twitter_agg_stream = (twitter_aggregation
    .writeStream
    .outputMode("append")
    .format("console")
    .start())

In [10]:
twitter_agg_stream.stop()
twitter_agg_stream.status

{'message': 'Stopped', 'isDataAvailable': False, 'isTriggerActive': False}

In [9]:
# Add the timestamp as key
twitter_aggregation = twitter_aggregation.withColumn('key', F.col('timestamp'))

# Send the data to kafka
(twitter_aggregation
    .selectExpr("CAST(key AS STRING) AS key", "to_json(struct(*)) AS value")
    .writeStream
    .format("kafka")
    .option("kafka.bootstrap.servers", "kafka:9092")
    .option("topic", "twitter-agg")
    .option("checkpointLocation", "checkpoints/twitter-agg")
    .start())

#### Crypto stream

In [10]:
# Define the timestamp format
timestampFormat = "dd-MM-yyyy HH:mm:ss"

# Create the schema of incoming data
crypto_schema = T.StructType([
    T.StructField('timestamp', T.TimestampType(), False),
    T.StructField('price', T.DoubleType(), False)
])

In [11]:
# Read kafka stream and subscribe to crypto topic
crypto_stream = (spark.readStream
          .format('kafka')
          .option('kafka.bootstrap.servers', 'kafka:9092')
          .option('startingOffsets', 'latest')
          .option('subscribe', 'crypto')
          .load()
          .select(F.col("key").cast("string"), \
                  F.from_json(F.col("value").cast("string"), crypto_schema, \
                  { "timestampFormat": timestampFormat }).alias("value")))

In [12]:
# Create streaming moving windows
crypto_aggregation = (crypto_stream
                     .select('value.*')
                     .withWatermark('timestamp', '5 seconds')
                     .groupBy(window('timestamp', '10 minutes', '2 minutes'))
                     .agg(avg('price').alias('price'))).select(F.col('window.end').alias('timestamp'), F.col('price'))

# Successfully ingested this stream

In [13]:
# Add the timestamp as key
crypto_aggregation = crypto_aggregation.withColumn('key', F.col('timestamp'))

# Send the data to kafka
(crypto_aggregation
    .selectExpr("CAST(key AS STRING) AS key", "to_json(struct(*)) AS value")
    .writeStream
    .format("kafka")
    .option("kafka.bootstrap.servers", "kafka:9092")
    .option("topic", "crypto-agg")
    .option("checkpointLocation", "checkpoints/crypto-agg")
    .start())

#### Read the crypto aggregation stream

In [14]:
# Create the schema of incoming aggregated crypto data
crypto_agg_schema = T.StructType([
    T.StructField('timestamp', T.TimestampType(), False),
    T.StructField('price', T.DoubleType(), False)
])

# Read the crypto aggregation stream
crypto_agg_stream = ((spark.readStream
          .format('kafka')
          .option('kafka.bootstrap.servers', 'kafka:9092')
          .option('startingOffsets', 'latest')
          .option('subscribe', 'crypto-agg')
          .load()
          .select(
              F.col("key").cast("string"), 
              F.from_json(F.col("value").cast("string"), crypto_agg_schema).alias("value")))
                     .select('value.*'))

crypto_agg_stream.printSchema()

root
 |-- timestamp: timestamp (nullable = true)
 |-- price: double (nullable = true)



#### Read the twitter aggregation stream

In [15]:
# Create the schema of incoming aggregated crypto data
twitter_agg_schema = T.StructType([
    T.StructField('timestamp', T.TimestampType(), False),
    T.StructField('sentiment', T.DoubleType(), False),
    T.StructField('n_tweets', T.IntegerType(), False)
])

# Read the twitter aggregation stream
twitter_agg_stream = ((spark.readStream
          .format('kafka')
          .option('kafka.bootstrap.servers', 'kafka:9092')
          .option('startingOffsets', 'latest')
          .option('subscribe', 'twitter-agg')
          .load()
          .select(
              F.col("key").cast("string"), 
              F.from_json(F.col("value").cast("string"), twitter_agg_schema).alias("value")))
                     .select('value.*'))

twitter_agg_stream.printSchema()

root
 |-- timestamp: timestamp (nullable = true)
 |-- sentiment: double (nullable = true)
 |-- n_tweets: integer (nullable = true)



#### Join the two streams

In [16]:
merged_stream = (crypto_agg_stream
                    .join(twitter_agg_stream, 'timestamp')
                    .withWatermark('timestamp', '1 seconds'))

In [17]:
# Write merged stream to memory
stream_reader = (merged_stream
                       .writeStream
                       .queryName('merged_stream')
                       .format('memory')
                       .start())

In [ ]:
stream_reader.stop()
stream_reader.status

In [ ]:
# Create dataframe for the merged stream
merged_dataframe = spark.sql('select * from merged_stream')

In [49]:
# Create ML pipeline
price_diff_transformer = PriceDiffTransformer()
time_transformer = TimeTransformer()
lr_estimator = LinearRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)

lr_pipeline = Pipeline(stages=[price_diff_transformer, time_transformer, lr_estimator])

In [50]:
# Fit pipeline on historic data (df = historic processed dataset from mongo database)
lr_model = lr_pipeline.fit(df)

In [52]:
# Transform the dataframe w/ merged stream using the fitted pipeline
tested_df = lr_model.transform(merged_dataframe)
tested_df.show(5)

+--------------------+--------------------+--------------------+
|               label|            features|          prediction|
+--------------------+--------------------+--------------------+
| 0.08521205187025771|[0.28216666666594...|-0.01304727055802...|
|-0.03269565217578929|[-0.2347857142849...|0.004784843384880111|
|   0.376347826088022|[0.15798571428513...|-0.11302319085549772|
| 0.17443478261066048|[0.11616111111106...| 0.18359062315473934|
|  0.2888695652181923|[0.12505994151979...|0.042201475695771706|
+--------------------+--------------------+--------------------+
only showing top 5 rows

